In [16]:
import numpy as np
import re

roll_number = re.compile(r'([1,2][0,5,6,7,8,9].\d*)|tezansahu')

class Post(object):
    def __init__(self,author,subject,raw_text):
#         print(author,subject)
        self.author = re.findall(roll_number,author)[0]
        self.subject = re.findall(roll_number,subject)[0]
        self.raw_text = raw_text
        self.text,self.mentions = self.get_text_mentions(raw_text)
    
    def __str__(self):
        return f"{self.author} wrote the following about {self.subject} : \n {self.text}"
 
    def get_text_mentions(self,raw_text):
        # TODO
        return raw_text, None
    def asdict(self):
        return {"Author":self.author,"Subject":self.subject,"Text":self.text}


class Person(object):
    def __init__(self,id,name,degree,dept,hostel,ibs):
        self.id = re.findall(roll_number,id)[0]
        self.name = name
        self.degree = degree
        self.dept = dept
        self.hostel = hostel
        self.ibs = ibs   # TODO see how to put into sql
        self.posts_auth = []
        self.posts_subj = []
        self.posts_tagged = []
    def __str__(self):
        
        ibs_str = ", ".join(list(self.ibs))
        return f"Name: {self.name}, Program: {self.degree}, Dept: {self.dept}, Hostel: {self.hostel}, Clubs Associated with: {ibs_str}"
    
    def compute_post_stats(self):
        self.auth_len = np.mean([len(x.text.split()) for x in self.posts_auth])
        self.subj_len = np.mean([len(x.text.split()) for x in self.posts_subj])
        
    def asdict(self):
        return {"Name":self.name,"id":self.id,"Degree":self.degree,"Department":self.dept,"Hostel":self.hostel,
                "IBs":self.ibs,"Gender":self.gender, "Posts Auth":[x.asdict() for x in self.posts_auth],
                "Posts Subj":[x.asdict() for x in self.posts_subj]
               ,"Posts Tagged":[x.asdict() for x in self.posts_tagged]}

def get_gender(name):
    if name.lstrip().startswith("Ms."):
        # print("Girl")
        return "Female"
    else:
        return "Male"
from bs4 import BeautifulSoup as soup

def get_pers(page,url):
    page = soup(page,"html.parser")
    personal_info = page.find("ul",class_="list-unstyled puser-details-list").find('li')

    name = personal_info.find('h2').text.split('\n')[0]
    # print(name)
    dets = personal_info.find_all('h5')#.text

    try:
        degree,dept,hostel = dets[0].text.split('|')
    except:
        degree,dept,hostel = None,None,None
    try:
        ibs = (dets[1].text.split('|'))
    except:
        ibs = []

    new_pers = Person(url,name,degree,dept,hostel,ibs)

    incoming_edges = page.find("div",id="myTabContent").find_all("div",class_="sub-post-container")
    # print(len(incoming_edges))
    for post in incoming_edges:
        parent = post.find_parent('div')
        if "tab-pane" not in parent["class"] or "impression" in parent["id"]:
            # This is done to make sure that only posts are taken in
            continue
        author = post.find("div",class_="user-details").find("a")
        author_id = author['href']
        author_name = author.text  # This is not strictly necessary, keeping it for now
        subject = author.find_next("div",class_="user-details").find("a")
        subject_id = subject['href']
        subject_name = subject.text

        post_text = post.find("p",class_="post-para").text
        new_post = Post(author_id,subject_id,post_text)
        if new_pers.id == new_post.subject:  # Handle the case of two people with the same name
            new_pers.posts_subj.append(new_post)
        elif new_pers.id == new_post.author:
            new_pers.posts_auth.append(new_post)

        else:   # Need to handle mentions separae
            new_pers.posts_tagged.append(new_post)

    new_pers.compute_post_stats()
    return new_pers
# print(new_pers.auth_len,new_pers.subj_len)

In [17]:
# In this cell I try to get the roll numbers of everyone from the xlsx
import pandas as pd
import requests
import json 

df = pd.read_excel('roll_list.xls',header=None)
roll_numbers = df.loc[:,1]
names = df.loc[:,2]
all_people = {}
for idx,roll in (enumerate(roll_numbers)):
#     if idx < 730:
#         continue

    try:
        roll = str(roll).strip()
        url = f"https://yearbook.sarc-iitb.org/student-{roll}/"
        # csrftoken = "LtXF0VdvU6UlShr5CPjbFdKQaFAoUt5C32GQaoIj1q0BmypIUPqH1WhP2wznt591"
        # cookies = dict(_ga_QHYLVSRXM2="GS1.1.1624347864.32.1.1624348240.0", _ga="GA1.1.793406901.1620407148", csrftoken=csrftoken, sessionid="uzeibeb0ggqa6zpilm4j9d332yicw00m")
        # page = requests.get(url,cookies=cookies)
        # print(roll)
        page = open(f"raw_data_sunday/{roll}.html","r").read()
        # file = open(f"raw_data_sunday/{roll}.html","w")
        # file.write(page.text)
        # file.close()
        # pers = get_pers(page.text,url)
        pers = get_pers(page,url)
        pers.gender = get_gender(names[idx])
        all_people[roll] = (pers.asdict())
#         file = open("raw_data.jsonl","a")
#         print(json.dumps(pers.asdict()),file=file)
#         file.close()
    except Exception as e: 
        print(f"Skipping {roll} because {e}")
        break

json.dump(all_people,open("raw_data_latest.json","w"))


Skipping 170100035 because 'NoneType' object has no attribute 'find'


In [11]:
# !pip uninstall -y beautifulsoup4
!pip install beautifulsoup4 == 4.8.0


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 97 kB 3.4 MB/s 


In [18]:
!mkdir raw_data_sunday

mkdir: cannot create directory ‘raw_data_sunday’: File exists


In [32]:
!pip uninstall -y bs4

Found existing installation: bs4 0.0.1
Uninstalling bs4-0.0.1:
  Successfully uninstalled bs4-0.0.1
